In [ ]:
import sys 
sys.path.append('../../../')
from data_tools.load_raw_cluster_data import LoadExperiment as le
from plot_tools.fig_preamble import * # imports numpy, matplotlib, Py2PDF, plot_helper_funcs, plot_figstyle_sheet
from analysis_tools.common import sqr_err
import GPy

ver=0
datapath = '../../DATA_v'+str(ver)+'_/' #v'+str(ver)+'/DATA_v'+str(ver)+'/'
datafile = 'GPR_pattern'
savefig2 = '../svg/paper_v0_'
figname2 = 'GPR_pattern'

'''Motivation: recast our measurement record as a pattern recognition problem and 
see if periodic kernel outperforms GPR

@article{roberts2013gaussian,
  title={Gaussian processes for time-series modelling.},
  author={Roberts, S and Osborne, M and Ebden, M and Reece, S and Gibson, N and Aigrain, S},
  journal={Philosophical transactions. Series A, Mathematical, physical, and engineering sciences},
  volume={371},
  number={1984},
  pages={20110550--20110550},
  year={2013}
}

Finding: In the limit of high p, small lengthscale, a periodic kernel looks a lot like an RBF.
'''

In [ ]:
test_case=19
variation=5
total_runs=20

dataobject = le(test_case, variation, 
                skip = 1,
                GPRP_load='No', GPRP_path = './',
                LKFFB_load = 'Yes', LKFFB_path = datapath,
                AKF_load='No', AKF_path = './',
                LSF_load = 'No', LSF_path = './')
shape = dataobject.LKFFB_macro_truth.shape
macro_truth_full = dataobject.LKFFB_macro_truth.reshape(shape[0]*shape[1], shape[2])

In [ ]:
##########################################################################################
# Run an example of pattern recognition using the following manually tuned initial conditions
##########################################################################################

# Test Case 19 Variation 5

start_msmt_rec = 0
min_chunk = 2000
max_chunk = 2050

msmt_var = 500000

sigma_0=1000.
period_0 =4000
length_scale_0=0.01

rbf_lengthscale=10.
rbf_variance=10.

qrbf_lengthscale=20.
qrbf_variance=50.
qper_lengthscale=0.001
qper_variance=50.
qper_period=4000

# Test Case 24 Variation 1, 7
# start_msmt_rec = 0
# min_chunk = 1950
# max_chunk = 2000

# msmt_var = 100

# sigma_0=100.
# period_0 =4000
# length_scale_0=0.01

# rbf_lengthscale=40.
# rbf_variance=100.

# qrbf_lengthscale=20.
# qrbf_variance=50.
# qper_lengthscale=0.001
# qper_variance=50.
# qper_period=4000



In [ ]:
# we take out a chunk of the measurement record and then create our training points, 
# truth, noisy data, and test points for GPR

take_a_chunk_out = range(min_chunk, max_chunk, 1)
macro_truth = np.delete(macro_truth_full, take_a_chunk_out,axis=1)
testpoints = np.arange(start_msmt_rec, shape[2], dtype=np.float32)
training_pts = np.delete(np.arange(shape[2], dtype=np.float32), take_a_chunk_out)[start_msmt_rec:,np.newaxis]


In [ ]:
# MAIN 


pattern_algo = ['RBF', 'PER', 'QUASIPER']
pattern_pred_list = np.zeros((len(pattern_algo), total_runs, testpoints.shape[0]))
pattern_truth_list = np.zeros((len(pattern_algo), total_runs, testpoints.shape[0]))
pattern_failed_list = np.zeros((len(pattern_algo), total_runs))
opt_params=[]

for idx_run in xrange(total_runs):
    
    pick_truth = int(np.random.uniform (low=0, high=shape[0]*shape[1]))
    print("I picked", pick_truth)
    noiseless_data = np.asarray(macro_truth[pick_truth,start_msmt_rec:], dtype=np.float32)
    noisy_data = noiseless_data
    noisy_data += dataobject.LKFFB_msmt_noise_variance*np.random.normal(size=noiseless_data.shape[0])


    for idx_ker in xrange(len(pattern_algo)):        

        # Reset
        KERNEL=0
        gauss=0
        exact=0
        m1=0

    #   Built model

        KERNEL = {'RBF': GPy.kern.RBF(1, lengthscale=rbf_lengthscale, variance=rbf_variance), 
                  'PER': GPy.kern.StdPeriodic(1, period=period_0, 
                                              lengthscale=length_scale_0,
                                              variance=sigma_0),
                  'QUASIPER': GPy.kern.RBF(1, 
                                           lengthscale=qrbf_lengthscale,
                                            variance=qrbf_variance)*GPy.kern.StdPeriodic(1, 
                                                                               period=qper_period, 
                                                                               lengthscale=qper_lengthscale,
                                                                               variance=qper_variance)}


        gauss = GPy.likelihoods.Gaussian(variance=msmt_var)
        exact = GPy.inference.latent_function_inference.ExactGaussianInference()

        try:
            m1 = GPy.core.GP(X=training_pts, 
                             Y=noisy_data[:, np.newaxis],
                             kernel= KERNEL[pattern_algo[idx_ker]], 
                             likelihood=gauss, 
                             inference_method=exact)
            print(m1)
            m1.optimize(messages=True)
            pattern_pred_list[idx_ker, idx_run, :] = m1.predict(testpoints[:, np.newaxis])[0].flatten()
            opt_params.append(m1.flattened_parameters)
            print("Optimisation complete!")
            print(m1)
            print
            print

        except:
            print("Unexpected error:", sys.exc_info()[0])
            opt_params.append(np.zeros(6))
            pattern_failed_list[idx_ker, idx_run] = idx_run
            print("Failed run")

        pattern_truth_list[idx_ker, idx_run, :] = macro_truth_full[pick_truth,start_msmt_rec:]
        
        
        np.savez(datapath+'GPR_pattern_RBF_PER', 
                 pattern_truth_list=pattern_truth_list,
                 pattern_failed_list=pattern_failed_list,
                 pattern_pred_list=pattern_pred_list,
                 testpoints=testpoints,
                 training_pts=training_pts,
                 opt_params=opt_params)

In [ ]:
opt_params

In [ ]:
# PLOT IT

start_msmt_rec=0
pick_run=int(np.random.uniform(low=0, high=1))

gs = gridspec.GridSpec(1, 1,
                       left=0.15, right=0.985, 
                       top=0.8, bottom=0.1, 
                       wspace=0.1, hspace=0.2)

fig1 = plt.figure(figsize=(cm2inch(16),cm2inch(16.)))

ax_patrn = fig1.add_subplot(gs[0,0])


ax_patrn.plot(np.arange(start_msmt_rec-2000, min_chunk-2000), 
              pattern_truth_list[0][pick_run, start_msmt_rec:min_chunk], 'k-', #lw=1.,
              np.arange(max_chunk-2000, shape[2]-2000), 
              pattern_truth_list[0][pick_run, max_chunk:shape[2]], 'k-', 
              np.arange(min_chunk-2000, max_chunk-2000), 
              pattern_truth_list[0][pick_run,min_chunk:max_chunk], 'k-', lw=1.)

noiseless_data = np.delete(np.asarray(pattern_truth_list[0][pick_run,start_msmt_rec:], dtype=np.float32), take_a_chunk_out)
noisy_data = noiseless_data + dataobject.LKFFB_msmt_noise_variance*np.random.normal(size=noiseless_data.shape[0])

ax_patrn.plot(training_pts-2000, noisy_data,
               '.', lw=1.0, c='slategrey', label='Msmts')

ax_patrn.plot(testpoints-2000, pattern_pred_list[0, pick_run,:], ':', 
              c='maroon', label='RBF', lw=1.0, alpha=0.5)

ax_patrn.plot(testpoints-2000, pattern_pred_list[1, pick_run, :], '--', 
              c='slateblue', label='Periodic', lw=1.0, alpha=0.5,
              markerfacecolor='None')

ax_patrn.plot(testpoints-2000, pattern_pred_list[2, pick_run, :], '-.', 
              c='green', label='RBF*Periodic', lw=1.0, alpha=0.5)

ax_patrn.tick_params(direction='in', which='both')
ax_patrn.margins(0.2)
ax_patrn.set(xlabel='', ylabel='')
ax_patrn = set_font_sizes(ax_patrn, fsize, Fsize)
ax_patrn.set_xlim([1950-2000, 2100-2000])
ax_patrn.minorticks_off()

plt.legend(loc=0)
plt.show()
plt.close()